In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/BiasMitigation/LNTL')
print('Changed the local path to....', os.getcwd())

Changed the local path to.... /content/drive/MyDrive/BiasMitigation/LNTL


In [ ]:
# Local scripts
! pip install import_ipynb

import import_ipynb

from models.Deeplab import deeplabv3
from models.SegNet import segnet
from models import biashead
from utils.utils_LNTL import logger_setting, Timer
from utils.utils_Deeplab import add_weight_decay
# Python
import time
import os
import math
import numpy as np
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
# Torch
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
# Colab 
from google.colab.patches import cv2_imshow


class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * 0.1

def grad_reverse(x):
    return GradReverse.apply(x)



class Trainer(object):
    def __init__(self, option, path):
        self.option = option
        self.path = path

        self._build_model()
        self._set_optimizer()
        self.logger = logger_setting(option.exp_name, option.save_dir, path, option.debug)

    def _build_model(self):
        self.n_color_cls = 8    # This is how many colour bins we have used 256/32

        # The bias input channels depend on fork placement 
        if self.option.network_type == 'Deeplab': # CAN WE JUST USE option.network_type HERE BECUASE OF LINE 31
            bias_input_channels = 1280  # The number of feature maps after the aspp concat step.
        elif self.option.network_type == 'Segnet':
            bias_input_channels = 64

        #self.net = deeplabv3.DeepLabV3(model_id=self.option.exp_name, project_dir = self.option.save_dir) changed from this becuase i changed the arguements in DeepLabV3
        self.net = deeplabv3.DeepLabV3() # Doesnt need number of classes as its hard coded at the top of the deeplab class
        # The path might have to be changed since we have a directory models with the models inside. Rather than a script with the models inside as this is being called.
        # Also the deeplab class calls Aspp and Resnet so check paths in those scripts. 
        self.pred_net_r = biashead.BiasPredictor(input_ch=bias_input_channels, num_classes=self.n_color_cls)
        self.pred_net_g = biashead.BiasPredictor(input_ch=bias_input_channels, num_classes=self.n_color_cls)
        self.pred_net_b = biashead.BiasPredictor(input_ch=bias_input_channels, num_classes=self.n_color_cls)


        with open( self.option.meta_dir + "/class_weights.pkl", "rb") as file: # (needed for python3)
            class_weights = np.array(pickle.load(file))
        class_weights = torch.from_numpy(class_weights)
        class_weights = Variable(class_weights.type(torch.FloatTensor)).cuda()


        self.loss = nn.CrossEntropyLoss(weight = class_weights)  # must add weights to this one to perform cross entropy relatively (a traffic light is as important as a building)
        self.color_loss = nn.CrossEntropyLoss()


        if self.option.cuda:
            self.net.cuda()
            self.pred_net_r.cuda()
            self.pred_net_g.cuda()
            self.pred_net_b.cuda()
            self.loss.cuda()
            self.color_loss.cuda()

    def _set_optimizer(self): # worth adding in ADAM here?
        if self.option.optimiser == 'ADAM':
            params_net = add_weight_decay(self.net, l2_value = self.option.weight_decay) 
            params_r = add_weight_decay(self.pred_net_r, l2_value = self.option.weight_decay) 
            params_g = add_weight_decay(self.pred_net_g, l2_value = self.option.weight_decay) 
            params_b = add_weight_decay(self.pred_net_b, l2_value = self.option.weight_decay) 
            #####
            self.optim = optim.Adam(params_net, lr=self.option.lr)
            self.optim_r = optim.Adam(params_r, lr=self.option.lr)
            self.optim_g = optim.Adam(params_g, lr=self.option.lr)
            self.optim_b = optim.Adam(params_b, lr=self.option.lr)
        elif self.option.optimiser == 'SGD':
            self.optim = optim.SGD(filter(lambda p: p.requires_grad, self.net.parameters()), lr=self.option.lr, momentum=self.option.momentum, weight_decay=self.option.weight_decay)
            self.optim_r = optim.SGD(self.pred_net_r.parameters(), lr=self.option.lr, momentum=self.option.momentum, weight_decay=self.option.weight_decay)
            self.optim_g = optim.SGD(self.pred_net_g.parameters(), lr=self.option.lr, momentum=self.option.momentum, weight_decay=self.option.weight_decay)
            self.optim_b = optim.SGD(self.pred_net_b.parameters(), lr=self.option.lr, momentum=self.option.momentum, weight_decay=self.option.weight_decay)

        #TODO: last_epoch should be the last step of loaded model
        lr_lambda = lambda step: self.option.lr_decay_rate ** (step // self.option.lr_decay_period)
        self.scheduler = optim.lr_scheduler.LambdaLR(self.optim, lr_lambda=lr_lambda, last_epoch=-1)
        self.scheduler_r = optim.lr_scheduler.LambdaLR(self.optim_r, lr_lambda=lr_lambda, last_epoch=-1)
        self.scheduler_g = optim.lr_scheduler.LambdaLR(self.optim_g, lr_lambda=lr_lambda, last_epoch=-1)
        self.scheduler_b = optim.lr_scheduler.LambdaLR(self.optim_b, lr_lambda=lr_lambda, last_epoch=-1)

    @staticmethod
    def _weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))
        elif classname.find('BatchNorm') != -1:
            m.weight.data.fill_(1.0)
            m.bias.data.zero_()

    def _initialization(self):
        self.net.apply(self._weights_init)


        if self.option.is_train and self.option.use_pretrain:
            if self.option.checkpoint is not None:
                self._load_model()
            else:
                print("Pre-trained model not provided")



    def _mode_setting(self, is_train=True):
        if is_train:
            self.net.train()
            self.pred_net_r.train()
            self.pred_net_g.train()
            self.pred_net_b.train()
        else:
            self.net.eval()
            self.pred_net_r.eval()
            self.pred_net_g.eval()
            self.pred_net_b.eval()



    def _train_step(self, data_loader, step):
        _lambda = 0.01    # should we make this an option?
        self._mode_setting(is_train=True) # Put in train mode
        batch_losses_bias = []   # Reset bias batch losses
        batch_losses_seg = []   # Reset segmentation batch losses

        for i, (images, bias_labels, label_imgs) in enumerate(data_loader):
            
            images = self._get_variable(images)
            bias_labels = self._get_variable(bias_labels)
            label_imgs = self._get_variable(label_imgs.type(torch.LongTensor))  # converts to pytorch variable, uses CUDA if GPU available. 

            # reset gradients 
            self.optim.zero_grad()
            self.optim_r.zero_grad()
            self.optim_g.zero_grad()
            self.optim_b.zero_grad()
            pred_map, softmax, bias_fork = self.net(images)


            # predict colors from feat_label. Their prediction should be uniform.
            _,pseudo_pred_r = self.pred_net_r(bias_fork)  # outputs x and p(x),  here we get the softmax output
            _,pseudo_pred_g = self.pred_net_g(bias_fork)
            _,pseudo_pred_b = self.pred_net_b(bias_fork)


            # loss for self.net - semantic segmentation
            loss_pred = self.loss(pred_map, label_imgs) #the had a torch.squeeze here previously

            loss_pseudo_pred_r = torch.mean(torch.sum(pseudo_pred_r*torch.log(pseudo_pred_r),1))  # manual cross entropy p(x)log(p(x)) missing the negative sign. 
            loss_pseudo_pred_g = torch.mean(torch.sum(pseudo_pred_g*torch.log(pseudo_pred_g),1))
            loss_pseudo_pred_b = torch.mean(torch.sum(pseudo_pred_b*torch.log(pseudo_pred_b),1))
            
            
            loss_pred_ps_color = (loss_pseudo_pred_r + loss_pseudo_pred_g + loss_pseudo_pred_b) / 3.
            loss = loss_pred + loss_pred_ps_color*_lambda
            
            # DEBUGGING: #####
            print('**************************')
            #print('pseudo_pred_r.... \n', pseudo_pred_r)
            print('pseudo_pred_r.shape =  ', pseudo_pred_r.shape)
            print('**************************')
            #print('loss_pseudo_pred_r  \n', loss_pseudo_pred_r)
            print('loss_pseudo_pred_r.shape = ', loss_pseudo_pred_r.shape)
            print('**************************')
            #print('loss_pred_ps_color \n', loss_pred_ps_color)
            print('loss_pred_ps_color.shape  = ', loss_pred_ps_color.shape)
            print('**************************')
            print('loss', loss)
            ###################
            # Create loss value for plotting
            loss_value = loss_pred.data.cpu().numpy()  # this creates a numpy array on the cpu of the loss tensor for plotting
            batch_losses_seg.append(loss_value) 

            loss.backward()
            self.optim.step()

            # Reset gradients for the next stage of training schema
            self.optim.zero_grad()
            self.optim_r.zero_grad()
            self.optim_g.zero_grad()
            self.optim_b.zero_grad()

            pred_map, softmax, bias_fork = self.net(images)
            feat_color = grad_reverse(bias_fork)
            
            pred_r,_ = self.pred_net_r(feat_color)  # outputs x, p(x),  this time we get the layer before the softmax
            pred_g,_ = self.pred_net_g(feat_color)
            pred_b,_ = self.pred_net_b(feat_color)

            # loss for rgb predictors
            loss_pred_r = self.color_loss(pred_r, bias_labels[:,0]) # colour_loss() is the cross entropy instance, bias_labels are the ground truths created in dataloaders, [:,0] strips out R component 
            loss_pred_g = self.color_loss(pred_g, bias_labels[:,1])
            loss_pred_b = self.color_loss(pred_b, bias_labels[:,2])

            loss_pred_color = loss_pred_r + loss_pred_g + loss_pred_b

            # Create loss value for plotting
            loss_value = loss_pred_color.data.cpu().numpy()  # this creates a numpy array on the cpu of the loss tensor for plotting
            batch_losses_bias.append(loss_value) 

            loss_pred_color.backward()
            self.optim.step()
            self.optim_r.step()
            self.optim_g.step()
            self.optim_b.step()

            if i % self.option.log_step == 0:
                msg = "[TRAIN] cls loss : %.6f, rgb : %.6f, MI : %.6f  (epoch %d.%02d)" \
                       % (loss_pred,loss_pred_color/3.,loss_pred_ps_color,step,int(100*i/data_loader.__len__()))
                self.logger.info(msg)
        
        return batch_losses_seg, batch_losses_bias 
    
    def _validate_step(self, data_loader, step):
        self._mode_setting(is_train=False)
        batch_losses_seg = []
        batch_losses_bias = []

        for i, (images, bias_labels, label_imgs, img_id) in enumerate(data_loader):
            with torch.no_grad():
                images = self._get_variable(images)
                bias_labels = self._get_variable(bias_labels)
                label_imgs = self._get_variable(label_imgs.type(torch.LongTensor))

                # Segmentation Head
                # self.optim.zero_grad() dont need becuase its with torch.no_grad()
                pred_map, softmax , bias_fork = self.net(images) 

                # Loss for self.net, semantic segmentation loss
                loss_pred = self.loss(pred_map, label_imgs)  # cross entropy of the final layer before softmax and the cityscapes black images
                #debugging
                print('computed loss_pred in _validate_step...',  loss_pred)
                # Create loss value for plotting
                loss_value = loss_pred.data.cpu().numpy()  # this creates a numpy array on the cpu of the loss tensor for plotting
                batch_losses_seg.append(loss_value) 
            
                # Bias Head
                pred_r,_ = self.pred_net_r(bias_fork)  # outputs x, p(x),  this time we get the layer before the softmax
                pred_g,_ = self.pred_net_g(bias_fork)
                pred_b,_ = self.pred_net_b(bias_fork)
                print('pred_r.shape...', pred_r.shape)
                print('pred_g.shape...', pred_g.shape)
                print('pred_b.shape...', pred_b.shape)
                #~~~~~
                print('pred_map.shape ......', pred_map.shape)
                print('bias_fork.shape ......', bias_fork.shape)
                print('softmax.shape ......', softmax.shape)
                print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
                print('step....', i)
                print('images.shape ......', pred_map.shape)
                print('bias_labels.shape ......', bias_labels.shape)
                print('label_imgs.shape ......', label_imgs.shape)
                print('bias_labels[:,0].shape....', bias_labels[:,0].shape )
                #~~~~~
                loss_pred_r = self.color_loss(pred_r, bias_labels[:,0]) # colour_loss() is the cross entropy instance, bias_labels are the ground truths created in dataloaders, [:,0] strips out R component 
                print('computed loss_pred_r in _validate_step...',  loss_pred_r)
                loss_pred_g = self.color_loss(pred_g, bias_labels[:,1])
                print('computed loss_pred_g in _validate_step...',  loss_pred_g)
                loss_pred_b = self.color_loss(pred_b, bias_labels[:,2])
                print('computed loss_pred_b in _validate_step...',  loss_pred_b)

                loss_pred_color = loss_pred_r + loss_pred_g + loss_pred_b

                # Create loss value for plotting
                loss_value = loss_pred_color.data.cpu().numpy()  # this creates a numpy array on the cpu of the loss tensor for plotting
                batch_losses_bias.append(loss_value) 

        
            # TODO: print elapsed time for iteration
            if i % self.option.log_step == 0:
                msg = "[VAL] cls loss : %.6f (epoch %d.%02d)" \
                    % (loss_pred,step,int(100*i/data_loader.__len__()))
                self.logger.info(msg)
               
        return batch_losses_seg, batch_losses_bias

    def _train_step_baseline(self, data_loader, step):
        
        self._mode_setting(is_train=True) # Put in train mode
        batch_losses = []   # Reset batch losses

        for i, (images, bias_labels, label_imgs) in enumerate(data_loader):
            
            # test_images = images.cpu().detach().numpy()
            # test_bias_labels = bias_labels.cpu().detach().numpy()
            # test_label_imgs = label_imgs.cpu().detach().numpy()
            # cv2_imshow(test_images) 
            # cv2_imshow(test_bias_labels)
            # cv2_imshow(test_label_imgs)

            images = self._get_variable(images)
            label_imgs = self._get_variable(label_imgs.type(torch.LongTensor))

            self.optim.zero_grad()
            pred_map, softmax, bias_fork = self.net(images) 

            # Loss for self.net, semantic segmentation loss
            loss_pred = self.loss(pred_map, label_imgs)  # cross entropy of the final layer before softmax and the cityscapes black images
            # Create loss value for plotting
            loss_value = loss_pred.data.cpu().numpy()  # this creates a numpy array on the cpu of the loss tensor for plotting
            batch_losses.append(loss_value) 
            
            # Optimiser step
            loss_pred.backward()
            self.optim.step() # gradient descent


            # TODO: print elapsed time for iteration
            if i % self.option.log_step == 0:
                msg = "[TRAIN] cls loss : %.6f (epoch %d.%02d)" \
                    % (loss_pred,step,int(100*i/data_loader.__len__()))
                self.logger.info(msg)
               
        return batch_losses

    def _validate_step_baseline(self, data_loader, step):
        self._mode_setting(is_train=False)
        batch_losses = []
        
        for i, (images, bias_labels, label_imgs, img_id) in enumerate(data_loader):
            with torch.no_grad():
                images = self._get_variable(images)
                label_imgs = self._get_variable(label_imgs.type(torch.LongTensor))

                self.optim.zero_grad()
                pred_map, _ , _ = self.net(images) 

                # Loss for self.net, semantic segmentation loss
                loss_pred = self.loss(pred_map, label_imgs)  # cross entropy of the final layer before softmax and the cityscapes black images
                # Create loss value for plotting
                loss_value = loss_pred.data.cpu().numpy()  # this creates a numpy array on the cpu of the loss tensor for plotting
                batch_losses.append(loss_value) 
            
            # TODO: print elapsed time for iteration
            if i % self.option.log_step == 0:
                msg = "[VAL] cls loss : %.6f (epoch %d.%02d)" \
                    % (loss_pred,step,int(100*i/data_loader.__len__()))
                self.logger.info(msg)
               
        return batch_losses
    
    # def _validate(self, data_loader):
    #     self._mode_setting(is_train=False)
    #     self._initialization()
    #     if self.option.checkpoint is not None:
    #         self._load_model()
    #     else:
    #         print("No trained model for evaluation provided")
    #         import sys
    #         sys.exit()

    #     num_test = 10000

    #     total_num_correct = 0.
    #     total_num_test = 0.
    #     total_loss = 0.
    #     for i, (images, bias_labels, label_imgs, img_id) in enumerate(data_loader):
            
    #         start_time = time.time()
    #         images = self._get_variable(images)
    #         bias_labels = self._get_variable(bias_labels)
    #         label_imgs = self._get_variable(label_imgs.type(torch.LongTensor))

    #         self.optim.zero_grad()
    #         pred_map, softmax, bias_fork = self.net(images)


    #         loss = self.loss(pred_map, torch.squeeze(label_imgs)) #again not sure about squeeze check deeplab train script. 
            
    #         batch_size = images.shape[0]
    #         total_num_correct += self._num_correct(pred_map, label_imgs, topk=1).data[0]
    #         total_loss += loss.data[0]*batch_size
    #         total_num_test += batch_size
               
    #     avg_loss = total_loss/total_num_test
    #     avg_acc = total_num_correct/total_num_test
    #     msg = "EVALUATION LOSS  %.4f, ACCURACY : %.4f (%d/%d)" % \
    #                     (avg_loss,avg_acc,int(total_num_correct),total_num_test)
    #     self.logger.info(msg)


    def _num_correct(self,outputs,labels,topk=1):
        _, preds = outputs.topk(k=topk, dim=1)
        preds = preds.t()
        correct = preds.eq(labels.view(1, -1).expand_as(preds))
        correct = correct.view(-1).sum()
        return correct
        

    def _accuracy(self, outputs, labels):
        batch_size = labels.size(0)
        _, preds = outputs.topk(k=1, dim=1)
        preds = preds.t()
        correct = preds.eq(labels.view(1, -1).expand_as(preds))
        correct = correct.view(-1).float().sum(0, keepdim=True)
        accuracy = correct.mul_(100.0 / batch_size)
        return accuracy


    def _save_model(self, step): # this requires the directory to already be created (this is done in backend setting in main script)
        checkpoint_dir = self.path + '/checkpoints'
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        torch.save({
            'step': step,
            'optim_state_dict': self.optim.state_dict(),
            'net_state_dict': self.net.state_dict()
        }, os.path.join(checkpoint_dir, 'checkpoint_epoch_%04d.pth' % (step+1)) )
        print('Checkpoint saved. Epoch : %d'%step)


    def _load_model(self):
        ckpt = torch.load(self.option.checkpoint)
        self.net.load_state_dict(ckpt['net_state_dict']) # this is how we saved them in save model method above
        self.optim.load_state_dict(ckpt['optim_state_dict'])

    def _plotter_function(self, epoch_losses, stage, head = 'seg' ):
        if stage == 'train':
           if head == 'seg':
                with open("%s/epoch_losses_train.pkl" % self.path, "wb") as file:
                    pickle.dump(epoch_losses, file)
                plt.figure(1)
                plt.plot(epoch_losses, "k^")
                plt.plot(epoch_losses, "k")
                plt.ylabel("loss")
                plt.xlabel("epoch")
                plt.title("train loss per epoch")
                plt.savefig("%s/epoch_losses_train.png" % self.path )
                plt.close(1)
           elif head == 'bias':
                with open("%s/epoch_losses_train_bias_head.pkl" % self.path, "wb") as file:
                    pickle.dump(epoch_losses, file)
                plt.figure(1)
                plt.plot(epoch_losses, "m^")
                plt.plot(epoch_losses, "m")
                plt.ylabel("loss")
                plt.xlabel("epoch")
                plt.title("train loss per epoch in bias head")
                plt.savefig("%s/epoch_losses_train_bias_head.png" % self.path )
                plt.close(1)
        
        elif stage == 'val':
           if head == 'seg':
                with open("%s/epoch_losses_val.pkl" % self.path, "wb") as file:
                    pickle.dump(epoch_losses, file)
                plt.figure(1)
                plt.plot(epoch_losses, "k^")
                plt.plot(epoch_losses, "k")
                plt.ylabel("loss")
                plt.xlabel("epoch")
                plt.title("val loss per epoch")
                plt.savefig("%s/epoch_losses_val.png" % self.path )
                plt.close(1)
           elif head == 'bias':
                with open("%s/epoch_losses_val_bias_head.pkl" % self.path, "wb") as file:
                    pickle.dump(epoch_losses, file)
                plt.figure(1)
                plt.plot(epoch_losses, "m^")
                plt.plot(epoch_losses, "m")
                plt.ylabel("loss")
                plt.xlabel("epoch")
                plt.title("val loss per epoch in bias head")
                plt.savefig("%s/epoch_losses_val_bias_head.png" % self.path )
                plt.close(1)

    def train(self, train_loader, val_loader=None):
        #self._initialization()
        #if self.option.checkpoint is not None:
        #    self._load_model()

        self._mode_setting(is_train=True) #sets the model in .train mode
        timer = Timer(self.logger, self.option.max_step)
        start_epoch = 0
        
        epoch_losses_train = []
        epoch_losses_val = []
        epoch_losses_train_seg = []
        epoch_losses_val_seg = []
        epoch_losses_train_bias = []
        epoch_losses_val_bias = []
        
        for step in range(start_epoch, self.option.max_step):
            if self.option.train_baseline:
                batch_losses = self._train_step_baseline(train_loader, step)
                epoch_loss = np.mean(batch_losses)
                print('Training epoch loss:....', epoch_loss)
                epoch_losses_train.append(epoch_loss)
                self._plotter_function(epoch_losses_train, stage = 'train')
                ######
                batch_losses = self._validate_step_baseline(val_loader, step)
                epoch_loss = np.mean(batch_losses)
                print('Validation epoch loss:....', epoch_loss)
                epoch_losses_val.append(epoch_loss)
                epoch_losses = self._plotter_function(epoch_losses_val, stage = 'val')
            else:
                print('epoch....', step)
                batch_losses_seg, batch_losses_bias = self._train_step(train_loader,step) #LNTL proceedure
                epoch_loss_seg = np.mean(batch_losses_seg)
                epoch_loss_bias = np.mean(batch_losses_bias)
                print('Training epoch loss (segmentation head / bias head):....', epoch_loss_seg, '  /  ' , epoch_loss_bias)
                epoch_losses_train_seg.append(epoch_loss_seg)
                epoch_losses_train_bias.append(epoch_loss_bias)
                # Plotting and save to pickle
                self._plotter_function(epoch_losses_train_seg, stage = 'train')
                self._plotter_function(epoch_losses_train_bias, stage= 'train', head = 'bias')
                ######
                batch_losses_seg, batch_losses_bias = self._validate_step(val_loader, step)
                epoch_loss_seg = np.mean(batch_losses_seg)
                epoch_loss_bias = np.mean(batch_losses_bias)
                print('Validation epoch loss (segmentation head / bias head):....', epoch_loss_seg, '  /  ' , epoch_loss_bias)
                epoch_losses_val_seg.append(epoch_loss_seg)
                epoch_losses_val_bias.append(epoch_loss_bias)
                # Plotting and save to pickle
                self._plotter_function(epoch_losses_val_seg, stage = 'val')
                self._plotter_function(epoch_losses_val_bias, stage = 'val', head = 'bias')
            
            self.scheduler.step()
            self.scheduler_r.step()
            self.scheduler_g.step()
            self.scheduler_b.step()

            if step == 1 or step % self.option.save_step == 0 or step == (self.option.max_step-1):
                self._save_model(step)


    def _get_variable(self, inputs):
        if self.option.cuda:
            return Variable(inputs.cuda()) #Is there a difference between Variable(inputs).cuda()??
        return Variable(inputs)

importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/SegNet/segnet.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/models/BiasPredictor.ipynb
importing Jupyter notebook from /content/drive/My Drive/BiasMitigation/LNTL/utils/utils_LNTL.ipynb
